In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
train_df = pd.read_csv('housing_price_train.csv')
print(train_df.head())

   ID  SquareFeet  Bedrooms  Bathrooms Neighborhood  YearBuilt          Price
0   1      1686.0         2          3        Rural     1950.0  244716.345651
1   2      1912.0         3          1        Urban     1970.0  213071.025103
2   3      1011.0         5          1        Urban     2010.0   53282.098098
3   4      2539.0         2          2        Rural     1956.0  248908.682853
4   5      2313.0         4          1        Urban     1966.0  263133.076957


In [10]:
train_df.isna().sum()

ID              0
SquareFeet      2
Bedrooms        0
Bathrooms       0
Neighborhood    1
YearBuilt       6
Price           0
dtype: int64

In [12]:
train_no_null_df = train_df.dropna()
print(train_no_null_df.shape)

(39991, 7)


In [14]:
one_hot_encoded = pd.get_dummies(train_no_null_df['Neighborhood'])
one_hot_encoded=one_hot_encoded.astype(int)
train_no_null_df = pd.concat([train_no_null_df, one_hot_encoded], axis=1)
print(train_no_null_df.head())

   ID  SquareFeet  Bedrooms  Bathrooms Neighborhood  YearBuilt          Price  \
0   1      1686.0         2          3        Rural     1950.0  244716.345651   
1   2      1912.0         3          1        Urban     1970.0  213071.025103   
2   3      1011.0         5          1        Urban     2010.0   53282.098098   
3   4      2539.0         2          2        Rural     1956.0  248908.682853   
4   5      2313.0         4          1        Urban     1966.0  263133.076957   

   Rural  Suburb  Urban  
0      1       0      0  
1      0       0      1  
2      0       0      1  
3      1       0      0  
4      0       0      1  


In [16]:
np.unique(train_no_null_df["Neighborhood"])

array(['Rural', 'Suburb', 'Urban'], dtype=object)

In [18]:
# from sklearn.preprocessing import LabelEncoder

# # Khai báo biến le làm nhiệm vụ encode (mã hóa)
# le = LabelEncoder()

# # Cho le biết những giá trị cần mã hóa
# # mẫu: le.fit(["paris", "paris", "tokyo", "amsterdam"])
# le.fit(train_no_null_df['Neighborhood'])
# list(le.classes_)

In [20]:
# Mã hóa biến Neighborhood từ categorical thành biến Neighborhood_encoded dạng numerical
# train_no_null_df['Neighborhood_encoded'] = le.transform(train_no_null_df['Neighborhood'])
# print(train_no_null_df.head())

In [22]:
X = train_no_null_df[['SquareFeet', 'Bedrooms', 'Bathrooms', 'Rural', 'Suburb', 'Urban', 'YearBuilt']]
y = train_no_null_df['Price']

In [24]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)

print(reg.coef_)
print(reg.intercept_)

[ 9.93309819e+01  4.94943059e+03  2.68509061e+03 -4.29923395e+02
 -9.87142720e+02  1.41706612e+03  2.90394121e+00]
-2786.7950643928198


In [26]:
test_df = pd.read_csv('housing_price_test.csv')
print(test_df.head())

      ID  SquareFeet  Bedrooms  Bathrooms Neighborhood  YearBuilt
0  40001        1121         5          2        Urban       2004
1  40002        1087         4          1        Urban       1976
2  40003        2396         2          2       Suburb       1993
3  40004        2687         5          1        Urban       1979
4  40005        2685         4          3        Urban       1999


In [28]:
one_hot_encoded = pd.get_dummies(test_df['Neighborhood'])
one_hot_encoded=one_hot_encoded.astype(int)
test_df = pd.concat([test_df, one_hot_encoded], axis=1)
print(test_df.head())

      ID  SquareFeet  Bedrooms  Bathrooms Neighborhood  YearBuilt  Rural  \
0  40001        1121         5          2        Urban       2004      0   
1  40002        1087         4          1        Urban       1976      0   
2  40003        2396         2          2       Suburb       1993      0   
3  40004        2687         5          1        Urban       1979      0   
4  40005        2685         4          3        Urban       1999      0   

   Suburb  Urban  
0       0      1  
1       0      1  
2       1      0  
3       0      1  
4       0      1  


In [30]:
# # Mã hóa biến Neighborhood từ categorical thành biến Neighborhood_encoded dạng numerical
# test_df['Neighborhood_encoded'] = le.transform(test_df['Neighborhood'])
# print(test_df.head())

X_test = test_df[['SquareFeet', 'Bedrooms', 'Bathrooms', 'Rural', 'Suburb', 'Urban', 'YearBuilt']]

y_pred = reg.predict(X_test)
print(y_pred)

[145917.13408975 134824.04915512 255279.69203265 ... 228675.27115887
 290525.26403451 191573.83560349]


In [32]:
results = []
for i in range(len(y_pred)):
    results.append((test_df['ID'].tolist()[i], y_pred[i]))
    
print(results[:5])

[(40001, 145917.13408975204), (40002, 134824.04915512382), (40003, 255279.6920326512), (40004, 298711.76258470374), (40005, 298991.93006731593)]


In [34]:
results_df = pd.DataFrame(results, columns=['ID', 'Price'])
results_df.to_csv('submission.csv', index=False)

In [36]:
import pickle # thư viện  giúp lưu trữ mô hình
filename = 'model_house.sav'
pickle.dump(reg, open(filename, 'wb'))

In [40]:
loaded_model = pickle.load(open(filename, 'rb'))
x = pd.DataFrame([[2000,3,4,0,1,0,200]])
print(type(x))
print(x.shape)
y_pred = loaded_model.predict(x)
print('Dựa đoán là:',y_pred)

<class 'pandas.core.frame.DataFrame'>
(1, 7)
Dựa đoán là: [221057.46842309]


C:\Users\PT\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
